***
## Parte III
O objetivo desta parte do trabalho é analisar o comportamento dos índices das tabelas do SGBD através
do exame e análise das tabelas de estatísticas para consultas SQL sobre as tabelas “movies”, “actors”,
“casting”, e sobre uma tabela criada com dados aleatórios. Esta tarefa deverá ser executada somente com
o PostgreSQL.

- _Isabella Almeida Macedo Daniel-22250544_
- _Lucas do Nascimento Silva-22250552_
- _Luis Felipe dos Santos Lima-22250554_

In [1]:

import psycopg2

conn = psycopg2.connect( 
    host="localhost",
    port="5432",
    database="tpchdb",
    user="tpch",
    password="test123"
)

cursor = conn.cursor()

print("Conexão estabelecida!")

Conexão estabelecida!


--------------------------------------------
### Tarefa 9 – Preparação de Tabela Exemplo
Criar uma tabela com uma chave simples e alguns dados de exemplo. Cada valor de chave é um número
incremental e está associado a com valores que variam de 0 até 10:

> `DROP TABLE IF EXISTS t;`
> 
> `CREATE TABLE t (k serial PRIMARY KEY, v integer);`
> 
> `INSERT INTO t(v)`
> 
> `SELECT trunc(random() * 10) FROM generate_series(1,100000);`

__Entrega__: Imprimir os valores das 10 primeiras tuplas da tabela, ordenando por k.



In [2]:

cursor.execute("DROP TABLE IF EXISTS t; CREATE TABLE t (k serial PRIMARY KEY, v integer); INSERT INTO t(v) SELECT trunc(random() * 10) FROM generate_series(1,100000);")

In [3]:

cursor.execute("SELECT * FROM t LIMIT 10 OFFSET 0;")

rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 0)
(2, 4)
(3, 2)
(4, 9)
(5, 8)
(6, 5)
(7, 0)
(8, 9)
(9, 0)
(10, 0)


--------------------------------------------
### Tarefa 10 – Páginas criadas
Verifique quantas páginas com blocos foram criadas para a tabela da Tarefa 11.
>`SELECT relname, relpages, reltuples FROM pg_class WHERE relname='t';`

__Entrega:__ Imprimir o resultado do comando SQL.



In [4]:

cursor.execute("""
    SELECT ceil(pg_total_relation_size('t') / current_setting('block_size')::numeric)
    FROM pg_class
    WHERE relname = 't'
""")

result = cursor.fetchone()
print("numero de páginas:", result[0])

numero de páginas: 722


--------------------------------------------
### Tarefa 11 – Blocos
Verifique quantos blocos foram efetivamente usados numa consulta

>`SELECT pg_sleep(1);`
>
>`\pset x on`
>
>`SELECT * FROM pg_stats WHERE relname='t';`
>
>`SELECT pg_stat_reset();`
>
>`\pset x off`

__Observação:__ Em algumas versões do PostgreSQL, o atributo é chamado de tablename em vez de
relname.

__Entrega:__ Imprimir o resultado do comando SQL.


In [5]:
cursor.execute("EXPLAIN (ANALYZE, BUFFERS) SELECT * FROM t ORDER BY k LIMIT 10")

result = cursor.fetchall()

for row in result:
    if "Buffers:" in row[0]:
        blocks_used = int(row[0].split("=")[1].split()[0])
        print("Total de blocos: ", blocks_used)
        break

Total de blocos:  3


--------------------------------------------
### Tarefa 12 – Índice
Crie um índice para o atributo ‘v’ e realize consultas e criação de índice

a) Qual o tempo gasto para realizar uma consulta para um valor (tendo a tabela 100000 tuplas)?



In [6]:
import time
cursor.execute("CREATE INDEX idx_v ON t(v);")

In [7]:
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo total: ", execution_time, " segundos")

Tempo total:  0.0005488395690917969  segundos


b) Qual o tempo gasto para recriar um índice para o atributo ‘v’?


In [8]:

cursor.execute("DROP INDEX idx_v;")
start_time = time.time()
cursor.execute("CREATE INDEX idx_v ON t(v);")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo total: ", execution_time, " segundos")

Tempo total:  0.028629779815673828  segundos


Remova a tabela ‘t’ e crie novamente com 1.0000.000 de tuplas

c) Qual o tempo gasto para realizar uma consulta para um valor específico?



In [9]:
cursor.execute("DROP TABLE IF EXISTS t;")
cursor.execute("CREATE TABLE t (k serial PRIMARY KEY, v integer); INSERT INTO t(v) SELECT trunc(random() * 10) FROM generate_series(1,1000000);")
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo total: ", execution_time, " segundos")

Tempo total:  0.015421390533447266  segundos


d) Qual o tempo gasto para recriar um índice para o atributo ‘v’?



In [10]:
cursor.execute("""
    SELECT EXISTS (
        SELECT 1
        FROM pg_indexes
        WHERE indexname = 'idx_v');
""")

index_exists = cursor.fetchone()[0]

if index_exists:
    cursor.execute("DROP INDEX idx_v;")
start_time = time.time()
cursor.execute("CREATE INDEX idx_v ON t(v);")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo toal: ", execution_time, " segundos")

Tempo toal:  0.21191048622131348  segundos


--------------------------------------------
### Tarefa 13 - Fill factor
Quando se cria um novo índice, nem toda entrada no bloco do índice é usada. Um espaço livre é deixado,
conforme o parâmetro fillfactor.

Crie novos índices usando fillfactor = 60, 80, 90 e 100. Analise o desempenho de suas consultas usando as
mesmas condições da Tarefa 14.

>`ALTER TABLE foo SET ( fillfactor = 50);`
>
> `VACUUM FULL foo;`

__Entrega:__ Relatório com o resultado das perguntas



In [11]:
cursor.execute("CREATE INDEX idx_60 ON t(v) WITH (fillfactor = 60);")
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo total: ", execution_time, " segundos")

Tempo total:  0.0003306865692138672  segundos


In [12]:
cursor.execute("CREATE INDEX idx_80 ON t(v) WITH (fillfactor = 80);")
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo total: ", execution_time, " segundos")

Tempo total:  0.0004343986511230469  segundos


In [13]:
cursor.execute("CREATE INDEX idx_90 ON t(v) WITH (fillfactor = 90);")
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo total: ", execution_time, " s:egundos")

Tempo total:  0.0005540847778320312  s:egundos


In [14]:
cursor.execute("CREATE INDEX idx_100 ON t(v) WITH (fillfactor = 100);")
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo total: ", execution_time, " segundos")

Tempo total:  0.0005002021789550781  segundos


--------------------------------------------
### Tarefa 15 - Utilize índices com ordem DESC
Repita os testes das Tarefas 12 e 13 usando índices descendentes. Avalie e registre na ficha

> `CREATE INDEX i ON t(v DESC NULLS FIRST);`

__Entrega:__ Relatório com o resultado da avaliação


In [15]:

cursor.execute("CREATE INDEX idx_v11 ON t(v DESC NULLS FIRST);")
cursor.execute("""
    SELECT ceil(pg_total_relation_size('t') / current_setting('block_size')::numeric)
    FROM pg_class
    WHERE relname = 't'
""")

result = cursor.fetchone()
print("Total de páginas:", result[0])

Total de páginas: 12799


In [16]:
# REFAZENDO TAREFA 12-ÍNDICES DECRESCENTES
# -------------------------------------------------
cursor.execute("EXPLAIN (ANALYZE, BUFFERS) SELECT * FROM t ORDER BY k LIMIT 10")

result = cursor.fetchall()

for row in result:
    if "Buffers:" in row[0]:
        blocks_used = int(row[0].split("=")[1].split()[0])
        print("Total de bloco: ", blocks_used)
        break

Total de bloco:  4


In [17]:
# REFAZENDO TAREFA 13-ÍNDICES DECRESCENTES
# -------------------------------------------------

# Criando um índice para o atributo 'v'
cursor.execute("CREATE INDEX idx_v13 ON t(v DESC NULLS FIRST);")

# Realizando uma consulta para um valor específico (100)
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("a) Tempo gasto para realizar a consulta: ", execution_time, " segundos")

# -------------------------------------------------
# Removendo o índice existente
cursor.execute("DROP INDEX idx_v13;")

# Recriando o índice para o atributo 'v'
start_time = time.time()
cursor.execute("CREATE INDEX idx_v13 ON t(v DESC NULLS FIRST);")
end_time = time.time()
execution_time = end_time - start_time
print("b) Tempo gasto para recriar o índice: ", execution_time, " segundos")

# -------------------------------------------------
# Removendo a tabela existente
cursor.execute("DROP TABLE IF EXISTS t;")

# Verificando a existência do índice
cursor.execute("""
# Criando novamente a tabela com 1.000.000 de tuplas
cursor.execute("CREATE TABLE t (k serial PRIMARY KEY, v integer); INSERT INTO t(v) SELECT trunc(random() * 10) FROM generate_series(1,1000000);")

# Realizando uma consulta para um valor específico (100)
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("c) Tempo gasto para realizar a consulta: ", execution_time, " segundos")

# -------------------------------------------------
# Verificando a existência do índice
cursor.execute("""
    SELECT EXISTS (
        SELECT 1
        FROM pg_indexes
        WHERE indexname = 'idx_v13');
""")
# Verificando a existência do índice
cursor.execute("""

index_exists = cursor.fetchone()[0]

if index_exists:
    # Removendo o índice existente
    cursor.execute("DROP INDEX idx_v13;")

# Recriando o índice para o atributo 'v'
start_time = time.time()
cursor.execute("CREATE INDEX idx_v13 ON t(v DESC NULLS FIRST);")
end_time = time.time()
execution_time = end_time - start_time
print("d) Tempo gasto para recriar o índice: ", execution_time, " segundos")

a) Tempo gasto para realizar a consulta:  0.00037288665771484375  segundos
b) Tempo gasto para recriar o índice:  0.2599203586578369  segundos
c) Tempo gasto para realizar a consulta:  0.017119884490966797  segundos
d) Tempo gasto para recriar o índice:  0.24817180633544922  segundos
